In [1]:
#imports
import json
import time
import random
import os
import numpy as np
import nltk
import matplotlib.pyplot as plt
import re
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn import decomposition
from twython import TwythonStreamer
from datetime import datetime
from wordcloud import WordCloud
from textblob import TextBlob
from nltk.tokenize import RegexpTokenizer
from stop_words import get_stop_words
from nltk.stem.porter import PorterStemmer
from gensim import corpora
import gensim

C:\Users\Tanvi\Anaconda2\lib\site-packages\gensim\utils.py:840: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


In [6]:
#variables and functions
tweets = []
x = random.randrange(5,10)
dt = str(datetime.now().time())[:8].translate(None,':')
kywrds=['Obama','Trump','Clinton']
jsonFiles = []
polObama,subObama = [],[]
polTrump,subTrump = [],[]
polClinton,subClinton=[],[]
obama,trump,clinton = '','',''
obamaCorpus,trumpCorpus,clintonCorpus=[],[],[]
jsonObamaFiles,jsonTrumpFiles,jsonClintonFiles = [],[],[]
def strip_non_ascii(string):
    ''' Returns the string without non ASCII characters'''
    stripped = (c for c in string if 0 < ord(c) < 127)
    return ''.join(stripped)


In [7]:
os.chdir('C:/Users/Tanvi/Downloads/Project')
os.getcwd()

'C:\\Users\\Tanvi\\Downloads\\Project'

In [8]:
#Streaming 200 tweets per connection
class MyStreamer(TwythonStreamer):

    def on_success(self, data):
        if 'text' in data:
            if data['lang'] == 'en':
                tweets.append(data)
                print 'received tweet #', len(tweets), data['text'][:100].encode('utf-8')
        
        #200 tweets per connection to be stored in a json file
        if len(tweets) >= 200:
            self.store_json()
            self.disconnect()
        
    def on_error(self, status_code, data):
        print status_code, data
        self.store_json()        
        self.disconnect()

    def store_json(self):
        with open('tweet_stream_{}_{}_{}.json'.format(dt, kywrds[wrd], i), 'w') as f:
            json.dump(tweets, f, indent=4)
            


In [ ]:

if __name__ == '__main__':

    with open('tanvi_twitter_credentials.json', 'r') as f:
        credentials = json.load(f)

    # create your own app to get consumer key and secret
    CONSUMER_KEY = credentials['CONSUMER_KEY']
    CONSUMER_SECRET = credentials['CONSUMER_SECRET']
    ACCESS_TOKEN = credentials['ACCESS_TOKEN']
    ACCESS_TOKEN_SECRET = credentials['ACCESS_TOKEN_SECRET'] 
    os.chdir('./json')
    for wrd in range(len(kywrds)):
        for i in range(1,51):
            stream  = MyStreamer(CONSUMER_KEY, CONSUMER_SECRET, ACCESS_TOKEN, ACCESS_TOKEN_SECRET)            
            stream.statuses.filter(track=kywrds[wrd])
            tweets=[]
            time.sleep(x)

In [9]:
#Sentiment Analysis
jsonObamaFiles,jsonTrumpFiles,jsonClintonFiles = [],[],[]
filepath = 'C:/Users/Tanvi/Downloads/Project/json'
for files in os.listdir(filepath):
    if files.endswith('json') and 'Obama' in files:
        jsonObamaFiles.append(files)
    elif files.endswith('json') and 'Trump' in files:
        jsonTrumpFiles.append(files)
    elif files.endswith('json') and 'Clinton' in files:
        jsonClintonFiles.append(files)
os.chdir(filepath)
for filename in jsonObamaFiles:
    infile = open(filename).read()
    content = json.loads(infile)
    for i in range(len(content)):        
        tweet = strip_non_ascii(content[i]['text'])
        obama += tweet + '\n'
        senObama = TextBlob(tweet)
        polObama.append(senObama.sentiment.polarity)
        subObama.append(senObama.sentiment.subjectivity)
    
for filename in jsonTrumpFiles:
    infile = open(filename).read()
    content = json.loads(infile)
    for i in range(len(content)):        
        tweet = strip_non_ascii(content[i]['text'])
        trump += tweet + '\n' 
        senTrump = TextBlob(tweet)
        polTrump.append(senTrump.sentiment.polarity)
        subTrump.append(senTrump.sentiment.subjectivity)
    
for filename in jsonClintonFiles:
    infile = open(filename).read()
    content = json.loads(infile)
    for i in range(len(content)):        
        tweet = strip_non_ascii(content[i]['text'])
        clinton += tweet + '\n' 
        senClinton = TextBlob(tweet)
        polClinton.append(senClinton.sentiment.polarity)
        subClinton.append(senClinton.sentiment.subjectivity)


print('Total number of tweets collected:{}\n'.format((len(jsonObamaFiles)+len(jsonTrumpFiles)+len(jsonClintonFiles))*len(content)))
print('Polarity:\nObama:{} Trump:{} Clinton:{}\n'.format(np.mean(polObama),np.mean(polTrump),np.mean(polClinton)))
print('Subjectivity:\nObama:{} Trump:{} Clinton:{}\n'.format(np.mean(subObama),np.mean(subTrump),np.mean(subClinton)))
print('Total no. of tweets:\nObama:{} Trump:{} Clinton:{}\n'.format(len(subObama),len(subTrump),len(subClinton)))


Total number of tweets collected:30200

Polarity:
Obama:0.0339125475215 Trump:0.059704368941 Clinton:0.0207804921994

Subjectivity:
Obama:0.312074594767 Trump:0.312719156887 Clinton:0.325309725985

Total no. of tweets:
Obama:10000 Trump:10200 Clinton:10000



In [ ]:
#Storing tweets in files
with open('ObamaTweets.txt', 'w') as f2:
    f2.write(obama)
with open('TrumpTweets.txt', 'w') as f2:
    f2.write(trump)
with open('ClintonTweets.txt', 'w') as f2:
    f2.write(clinton)

In [49]:
#WordClouds
os.chdir('C:/Users/Tanvi/Downloads/Project')
stopwords = nltk.corpus.stopwords.words('english')
stopwords.append(u'https')
stopwords.append(u'http')
stopwords.append(u'rt')

wcObama = WordCloud(max_font_size=40,stopwords=stopwords).generate(obama.lower())
wcObama.to_file("obama1.png")
plt.figure()
plt.imshow(wcObama)
plt.axis("off")
plt.show()

wcTrump = WordCloud(max_font_size=40,stopwords=stopwords).generate(trump.lower())
wcTrump.to_file("trump1.png")
plt.figure()
plt.imshow(wcTrump)
plt.axis("off")
plt.show()

wcClinton = WordCloud(max_font_size=40,stopwords=stopwords).generate(clinton.lower())
wcClinton.to_file("clinton1.png")
plt.figure()
plt.imshow(wcClinton)
plt.axis("off")
plt.show()

In [ ]:
#Streaming tweets for specific locations
kywrds=['SFO','NYC','DAL','LA','CHI','DC','ATL']
locations = ['-122.75,36.8,-121.75,37.8',
             '-74,40,-73,41',
             '-96.89,32.66,-96.67,32.87',
             '-118.79,32.64,-116.97,34.90',
             '-87.66,41.83,-87.63,42.01',
             '-77.03,38.86,-76.97,38.94',
             '-85.02,33.37,-83.81,34.22']
os.chdir('./json1')
if __name__ == '__main__':
   
    for wrd in range(7):
        for i in range(1,11):
            stream  = MyStreamer(CONSUMER_KEY, CONSUMER_SECRET, ACCESS_TOKEN, ACCESS_TOKEN_SECRET)            
            stream.statuses.filter(locations=locations[wrd])
            tweets=[]
            time.sleep(x)

In [10]:
#Sentiment Analysis for those locations
jsonFiles = []
polObama,subObama = [],[]
polTrump,subTrump = [],[]
polClinton,subClinton=[],[]
obama,trump,clinton = '','',''
kywrds=['ATL','CHI','DAL','DC','LA','SFO','NYC']
filepath = 'C:/Users/Tanvi/Downloads/Project/json1'

os.chdir(filepath)
for files in os.listdir(filepath):
    if files.endswith(".json") and ('Obama' not in files) and ('Trump' not in files ) and ('Clinton' not in files):
        jsonFiles.append(files)
jsonFiles.sort()
kywrds.sort()
j,k=0,0
for filename in jsonFiles:
    infile = open(filename).read()
    content = json.loads(infile)
    #to be run for the number of tweets collected per json file   
    k += 1
    for i in range(len(content)):        
        token = content[i]['text']
        
        if 'obama' in token.lower():
            token = strip_non_ascii(token)
            obama += token + '\n'
            senObama = TextBlob(token)
            polObama.append(senObama.sentiment.polarity)
            subObama.append(senObama.sentiment.subjectivity)
        elif ('trump' or 'donald') in token.lower():
            token = strip_non_ascii(token)
            trump += token + '\n'
            senTrump = TextBlob(token)
            polTrump.append(senTrump.sentiment.polarity)
            subTrump.append(senTrump.sentiment.subjectivity)
        elif ('clinton' or 'hillary') in token.lower():
            token = strip_non_ascii(token)
            clinton += token + '\n'
            senClinton = TextBlob(token)
            polClinton.append(senClinton.sentiment.polarity)
            subClinton.append(senClinton.sentiment.subjectivity)
    with open('{}.txt'.format(kywrds[j]), 'a') as f2:
        f2.write(obama)
        f2.write(trump)
        f2.write(clinton)
    if k % 10 == 0:
        j += 1
print('Total number of tweets collected:{}\n'.format(len(jsonFiles)*len(content)))
print('Polarity:\nObama:{} Trump:{} Clinton:{}\n'.format(np.mean(polObama),np.mean(polTrump),np.mean(polClinton)))
print('Subjectivity:\nObama:{} Trump:{} Clinton:{}\n'.format(np.mean(subObama),np.mean(subTrump),np.mean(subClinton)))

Total number of tweets collected:14000

Polarity:
Obama:0.0346672620324 Trump:0.080184319246 Clinton:0.0251427085986

Subjectivity:
Obama:0.374303683738 Trump:0.357106461748 Clinton:0.421674199983



In [ ]:
#Wordclouds for each candidate of each location
for wrd in kywrds:
    infile = open('{}.txt'.format(wrd)).read()
    if 'obama' in infile.lower():
        wcObama = WordCloud(max_font_size=40,stopwords=stopwords).generate(infile.lower())
        wcObama.to_file("obama{}.png".format(wrd))        
        plt.figure()
        plt.imshow(wcObama)
        plt.axis("off")
        plt.show()
    else:
        print('No tweets of obama in {}'.format(wrd))
    if 'trump' in infile.lower(): 
        wcTrump = WordCloud(max_font_size=40,stopwords=stopwords).generate(infile.lower())
        wcTrump.to_file("trump{}.png".format(wrd))        
        plt.figure()
        plt.imshow(wcTrump)
        plt.axis("off")
        plt.show()
    else:
        print('No tweets of trump in {}'.format(wrd))
    if 'clinton' in infile.lower():
        wcClinton = WordCloud(max_font_size=40,stopwords=stopwords).generate(infile.lower())
        wcClinton.to_file("clinton{}.png".format(wrd))
        plt.figure()
        plt.imshow(wcClinton)
        plt.axis("off")
        plt.show()
    else:
        print('No tweets of clinton in {}'.format(wrd))

In [95]:
#Preparing corpus for NMF
jsonFiles = []
polObama,subObama = [],[]
polTrump,subTrump = [],[]
polClinton,subClinton=[],[]
obama,trump,clinton = '','',''
def parse_text(text): #funtion to clean tweet data
    text = re.sub(r"http\S+", "", text) 
    text = re.sub(r"@\S+", "", text)    
    text = re.sub(r"#\S+", "", text)    
    text = re.sub(r'[-.#/?@🔪!,":;()]',' ',text.lower().replace('\n', ' '))
    text = re.sub(r'[\']','',text) 
    text = re.sub(r"\d", "", text) 
    text = re.sub(r'https|2016|hillary|clinton|hillaryclinton|donald|trump|Trump|donaldtrump|obama2016|barackobama|barack|obama|https|2016|rt','',text)
    text = ' '.join(text.split())  
    return text

os.chdir('C:/Users/Tanvi/Downloads/Project/json')
jsonObamaFiles,jsonObamaFiles,jsonClintonFiles=[],[],[]
for files in os.listdir('../json'):
    if files.endswith('json') and 'Obama' in files:
        jsonObamaFiles.append(files)
    elif files.endswith('json') and 'Trump' in files:
        jsonTrumpFiles.append(files)
    elif files.endswith('json') and 'Clinton' in files:
        jsonClintonFiles.append(files)
for filename in jsonObamaFiles:
    infile = open(filename).read()
    content = json.loads(infile)
    for i in range(len(content)):        
        tweet = parse_text(content[i]['text'])
        obama += tweet + '\n'
    obamaCorpus.append(obama)
    obama=''
for filename in jsonTrumpFiles:
    infile = open(filename).read()
    content = json.loads(infile)
    for i in range(len(content)):        
        tweet = parse_text(content[i]['text'])
        trump += tweet + '\n'
    trumpCorpus.append(trump)
    trump = ''
for filename in jsonClintonFiles:
    infile = open(filename).read()
    content = json.loads(infile)
    for i in range(len(content)):        
        tweet = parse_text(content[i]['text'])
        clinton += tweet + '\n'
    clintonCorpus.append(clinton)
    clinton=''

In [96]:
#Running NMF for tweets of each candidate
def corp(corpText):
    dtm = vectorizer.fit_transform(corpText) #obamaCorpus is a list of files having tweets.
    print(dtm.shape)
    
    num_terms = len(vectorizer.vocabulary_)
    terms = [""] * num_terms
    for term in vectorizer.vocabulary_.keys():
    	terms[ vectorizer.vocabulary_[term] ] = term
    
    num_topics = 20
    num_top_words = 10
    clf = decomposition.NMF(n_components = num_topics, random_state=2)
    doctopic = clf.fit_transform(dtm)
    
    topic_words = []
    for topic in clf.components_:
        word_idx = np.argsort(topic)[::-1][0:num_top_words]
        #print (word_idx)    
        topic_words.append([terms[i] for i in word_idx])
    
    for t in range(len(topic_words)):
        print("Topic {}: {}".format(t, ' '.join(topic_words[t][:20])))
    topic_words = []
    
vectorizer = TfidfVectorizer(stop_words='english', min_df=2)
corp(obamaCorpus)
corp(trumpCorpus)
corp(clintonCorpus)

(50, 4138)
Topic 0: biden president joe says past password losers know dont whats
Topic 1: mothers kids forced apologize whos like president look biden yorker
Topic 2: issued warning sick rioters completely just media silent president amp
Topic 3: biden just president left needs bag bathroom cheetos case powder
Topic 4: cutest couple michelle biden president amp just joe disappear white
Topic 5: yeah presi daquan outta officer car step used biden president
Topic 6: owns phone president biden just joe degree says dont shares
Topic 7: predicted baba vanga president teacher gorilla fired calling american biden
Topic 8: fundamental just stay photos years difference biden amp joe merkel
Topic 9: president news fake freedoms problem amp democratic says located thankful
Topic 10: biden president just world merkel amp joe white michelle decal
Topic 11: biden president amp joe right new genocide just years dont
Topic 12: biden president wwc joe amp ryan like russia years problems
Topic 13: pres

In [4]:
#Running LDA
tokenizer = RegexpTokenizer(r'\w+')
twitter_words= ['https','co','http','rt','RT','u','t','amp',]
en_stop = get_stop_words('en')+twitter_words
p_stemmer = PorterStemmer()
kywrds = ['Obama','Trump','Clinton']

for wrd in kywrds:
    infile = open('C:\\Users\\Tanvi\\Downloads\\Project\\json\\{}Tweets.txt'.format(wrd)).readlines()
    texts = []

    for i in infile:
        raw = i.lower()
        tokens = tokenizer.tokenize(raw)
        stopped_tokens = [i for i in tokens if not i in en_stop]
        stemmed_tokens = [p_stemmer.stem(i) for i in stopped_tokens]
        texts.append(stemmed_tokens)
    dictionary = corpora.Dictionary(texts)
    corpus = [dictionary.doc2bow(text) for text in texts]

    # generate LDA model
    ldamodel = gensim.models.ldamodel.LdaModel(corpus, num_topics=20, id2word = dictionary, passes=20)
    print(ldamodel.print_topics())



[(0, u'0.090*"obama" + 0.048*"trump" + 0.031*"russia" + 0.029*"go" + 0.028*"now" + 0.027*"stand" + 0.019*"presid" + 0.019*"say" + 0.018*"hope" + 0.018*"pre"'), (1, u'0.111*"trump" + 0.071*"obama" + 0.046*"protest" + 0.041*"anti" + 0.023*"march" + 0.019*"two" + 0.018*"foxnew" + 0.012*"chief" + 0.012*"advisor" + 0.011*"s"'), (2, u'0.053*"yeah" + 0.043*"need" + 0.037*"hillari" + 0.037*"biden" + 0.033*"case" + 0.033*"obama" + 0.028*"powder" + 0.028*"no" + 0.026*"offic" + 0.019*"pardon"'), (3, u'0.060*"obama" + 0.053*"trump" + 0.020*"presid" + 0.018*"put" + 0.018*"crime" + 0.017*"potu" + 0.017*"fuck" + 0.015*"car" + 0.014*"america" + 0.013*"hate"'), (4, u'0.155*"biden" + 0.042*"say" + 0.034*"obama" + 0.034*"loser" + 0.030*"left" + 0.024*"bag" + 0.023*"whisper" + 0.023*"cheeto" + 0.023*"bathroom" + 0.020*"hand"'), (5, u'0.078*"obama" + 0.018*"vote" + 0.016*"michel" + 0.014*"pleas" + 0.014*"call" + 0.013*"pardon" + 0.013*"democrat" + 0.013*"republican" + 0.012*"2" + 0.012*"clinton"'), (6, u'0